### nvdiffrecをcolaboratory上で動かす
前提：  
* Google driveに5GB以上の空きがある
* [colmap2poses.py](https://gist.github.com/Sazoji/e20835d652c51f305ce328342af7fefd)を実行していること
* ランタイム設定は「GPU」もしくは「プレミアムGPU」

### 1. Google Driveとファイルを同期
drive/project_folder/target_folder（imagesやmaskを含んだフォルダ） というフォルダ階層で用意します。

In [ ]:
from google.colab import drive
import os

# @markdown ### Path
project_folder = 'nvdiffrec_on_colab' #@param {type:"string"}
target_folder = '' #@param {type:"string"}

drive.mount('/content/drive')

colab_dir = f'drive/MyDrive/{project_folder}'
os.chdir(colab_dir)
print('you are in' + os.getcwd() + ' now')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
you are in/content/drive/MyDrive/nvdiffrec now


### 2. 関連ライブラリのインストール
nvdiffrecをcloneしていない場合は「nvdiffrec_install」にチェックを入れます。  
完了後、上タブのランタイムから「ランタイムを再起動」してください。その後、再度1.を実行してください。以降、2.の実行は不要です。

In [ ]:
# @markdown ### Setup
nvdiffrec_install = False #@param {type:"boolean"}

!pip install ninja  glfw xatlas
!pip install git+https://github.com/NVlabs/nvdiffrast/
!pip install --global-option="--no-networks" git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch
!imageio_download_bin freeimage
!pip install rembg
if nvdiffrec_install:
  !git clone https://github.com/NVlabs/nvdiffrec.git
else:
  pass
print('ランタイムを再起動してください')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 7.0 MB/s 
     |████████████████████████████████| 207 kB 65.2 MB/s 
     |████████████████████████████████| 206 kB 28.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVlabs/nvdiffrast/ to /tmp/pip-req-build-zucym79g
  Running command git clone -q https://github.com/NVlabs/nvdiffrast/ /tmp/pip-req-build-zucym79g
  Created wheel for nvdiffrast: filename=nvdiffrast-0.3.0-py3-none-any.whl size=140212 sha256=797cf06792667911bfc4bc9d520f64c594b5b209c9558bd8e30832e7496faa72
  Stored in directory: /tmp/pip-ephem-wheel-cache-_2ansu90/wheels/36/6d/8c/1e420551a8e968c4610e89456cef5929624d5e3cf64a7fa546
Successfully built nvdiffrast
/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of -

ランタイムを再起動してください


### 3. 前処理
* use_EISeg_mask  
　マスク画像の作成にEISegを使った場合のみチェック。EISegを使用して作成したマスクを、モノクロに変換して再配置します。
* train_resolution  
　訓練画像の解像度です。

In [ ]:
# @markdown ### Preprocessing

# 画像が見切れていることが悪いのでは？という仮説がある。ので、見切れている画像は削除する
import cv2
import glob, shutil, pathlib, os
import numpy as np

org_format = 'jpg'
identifier = '_pseudo.png'
input_dir = pathlib.Path(f'../{target_folder}/images/label')
out_dir = f'../{target_folder}/masks'
os.makedirs(out_dir, exist_ok=True)

use_EISeg_mask = False #@param {type:"boolean"}
if use_EISeg_mask:
    for f in input_dir.glob(f'**/*{identifier}'):
        image = cv2.imread(str(f))

        # 指定色
        base_color = (0, 0, 0)

        # 変更後の色
        change_color = (255, 255, 255)

        h, w = image.shape[:2]
        for i in range(h):
            for j in range(w):
                b, g, r = image[i, j]
                if (b, g, r) !=  base_color:
                    image[i, j] = change_color
        cv2.imwrite(f'{out_dir}/{str(f).split(identifier)[0].split("/")[-1]}.{org_format}', image)

# view_imgs.txtにないイメージを削除
base_dir = f'{target_folder}'
usable_files_list = []
defect_directory = os.path.join(base_dir, 'unused_images').replace('\\', '/')
defect_image_directory = os.path.join(defect_directory, 'images').replace('\\', '/')
defect_mask_directory = os.path.join(defect_directory, 'mask').replace('\\', '/')
dirs = [defect_directory, defect_image_directory, defect_mask_directory]

for d in dirs:
    if not os.path.isdir(d):
        os.makedirs(d)
with open(os.path.join(base_dir, 'view_imgs.txt')) as fp:
    for line in fp:
        usable_files_list.append(line.strip())

all_img = glob.glob(f"{base_dir}/images/*")
all_mask = glob.glob(f"{base_dir}/masks/*")
temp_all_img = []
temp_all_mask = []
unmatch_file_list = []

for idx, i in enumerate(all_mask):
    # 画像の端が見切れているか
    img = cv2.imread(i, cv2.IMREAD_COLOR)
    h,w = img.shape[:2]
    img_trim = img[1:h-1,1:w-1]
    img_trim = np.pad(img_trim, [1, 1], "constant")
    difference = (img - img_trim[:, :, 1:-1]).sum()
    if difference > 0:
        try:
            shutil.move(i, defect_mask_directory)  
        except shutil.Error:
            os.unlink(i)
        unmatch_file_list.append(os.path.split(i)[1])
        continue
    if os.path.split(i)[1] not in usable_files_list:
        try:
            shutil.move(i, defect_mask_directory)
        except shutil.Error:
            os.unlink(i)
    else:
        temp_all_mask.append(i)

for idx, i in enumerate(all_img):
    if (os.path.split(i)[1] not in usable_files_list) or (os.path.split(i)[1] in unmatch_file_list):
        try:
            shutil.move(i, defect_image_directory)
        except shutil.Error:
            os.unlink(i)
    else:
        temp_all_img.append(i)

all_img = temp_all_img
all_mask = temp_all_mask

print('copy to data/nerd/...')
shutil.copytree(target_folder, f'nvdiffrec/data/nerd/{target_folder}')

In [ ]:
%cd nvdiffrec/data/nerd

from pkg_resources import ResolutionError
# scale_imagesにクロップ処理を追加
import cv2
import os
import glob
import imageio
import shutil

import numpy as np
import torch
import torchvision

# @markdown ### Crop
clop_resolution = 1080

datasets = [target_folder]
folders  = ['images', 'masks']

# クロップを適用する
def get_moments_area(path, clopsize=[clop_resolution, clop_resolution]):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    m = cv2.moments(img, False)
    x, y = int(m["m10"]/m["m00"]), int(m["m01"]/m["m00"])
    return [[x - int(clopsize[0]/2), y - int(clopsize[1]/2)], [x + int(clopsize[0]/2), y + int(clopsize[1]/2)]]

def clop_moments_area(img, area, clopsize=[clop_resolution, clop_resolution]):
    # マージン追加
    y, x = img.shape[:2]
    min_x, min_y = area[0]
    max_x, max_y = area[1]
    if min_x < 0:
        max_x = clopsize[0]
        min_x = 0
    elif max_x > x:
        diff = max_x - x
        max_x -= diff
        min_x -= diff
    if min_y < 0:
        max_y = clopsize[1]
        min_y = 0
    elif max_y > y:
        diff = max_y - y
        max_y -= diff
        min_scalar_type -= diff
    # print(min_x, min_y, max_x, max_y)
    img = img[min_y:max_y, min_x:max_x, :]
    return img

for dataset in datasets:
    # 被写体の写っている箇所のみクロップ
    for mask in glob.glob(os.path.join(dataset, 'masks', '*')):
        filename = os.path.split(mask)[-1]
        img = imageio.imread(mask, pilmode='RGB').astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        moments_area = get_moments_area(mask)
        img = clop_moments_area(img, moments_area)
        cv2.imwrite(mask, img)
        # cv2_imshow(img)
        org = os.path.join(dataset, 'images', filename)
        img = imageio.imread(org, pilmode='RGB').astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = clop_moments_area(img, moments_area)
        cv2.imwrite(org, img)
        # cv2_imshow(img)

            
%cd ../../

/content/drive/MyDrive/nvdiffrec/nvdiffrec/data/nerd
/content/drive/MyDrive/nvdiffrec/nvdiffrec


In [ ]:
# nvdiffrec内のscale_images.pyとほぼ同様
%cd nvdiffrec/data/nerd

from pkg_resources import ResolutionError
# scale_imagesにクロップ処理を追加
import cv2
import os
import glob
import imageio
import shutil

import numpy as np
import torch
import torchvision

# @markdown ### Scaling
train_resolution = 512 #@param {type:"slider", min:256, max:2048, step:256}
resolution = [train_resolution, train_resolution]

datasets = [target_folder]
folders  = ['images', 'masks']

for dataset in datasets:
    dataset_rescaled = dataset + "_rescaled"
    os.makedirs(dataset_rescaled, exist_ok=True)
    shutil.copyfile(os.path.join(dataset, "poses_bounds.npy"), os.path.join(dataset_rescaled, "poses_bounds.npy"))
    for folder in folders:
        os.makedirs(os.path.join(dataset_rescaled, folder), exist_ok=True)
        files = glob.glob(os.path.join(dataset, folder, '*.jpg')) + glob.glob(os.path.join(dataset, folder, '*.JPG'))
        for file in files:
            print(file)
            img = torch.tensor(imageio.imread(file, pilmode='RGB').astype(np.float32) / 255.0)
            img = img[None, ...].permute(0, 3, 1, 2)
            rescaled_img = torch.nn.functional.interpolate(img, resolution, mode='area')
            rescaled_img = rescaled_img.permute(0, 2, 3, 1)[0, ...]
            out_file = os.path.join(dataset_rescaled, folder, os.path.basename(file))
            imageio.imwrite(out_file, np.clip(np.rint(rescaled_img.numpy() * 255.0), 0, 255).astype(np.uint8))
            
%cd ../../

/content/drive/MyDrive/nvdiffrec/nvdiffrec/data/nerd
Chair_/images/image_076.jpg
Chair_/images/image_090.jpg
Chair_/images/image_070.jpg
Chair_/images/image_062.jpg
Chair_/images/image_045.jpg
Chair_/images/image_061.jpg
Chair_/images/image_097.jpg
Chair_/images/image_057.jpg
Chair_/images/image_046.jpg
Chair_/images/image_055.jpg
Chair_/images/image_054.jpg
Chair_/images/image_053.jpg
Chair_/images/image_058.jpg
Chair_/images/image_048.jpg
Chair_/images/image_081.jpg
Chair_/images/image_060.jpg
Chair_/images/image_059.jpg
Chair_/images/image_073.jpg
Chair_/images/image_056.jpg
Chair_/images/image_047.jpg
Chair_/images/image_077.jpg
Chair_/masks/image_045.jpg
Chair_/masks/image_046.jpg
Chair_/masks/image_047.jpg
Chair_/masks/image_048.jpg
Chair_/masks/image_053.jpg
Chair_/masks/image_054.jpg
Chair_/masks/image_055.jpg
Chair_/masks/image_056.jpg
Chair_/masks/image_057.jpg
Chair_/masks/image_058.jpg
Chair_/masks/image_059.jpg
Chair_/masks/image_060.jpg
Chair_/masks/image_061.jpg
Chair_/m

### 4. 実行
configを設定して実行します。設定可能な項目は[nvdiffrecのリポジトリ](https://github.com/NVlabs/nvdiffrec)を参照してください。  
下記の設定項目はほんの一部になっています。

In [ ]:
import json

config_path = f'configs/{target_folder}_rescaled.json'

# @markdown ### Config
ref_mesh = f"data/nerd/{target_folder}_rescaled"
random_textures = True
iteration = 5000 #@param {type:"slider", min:0, max:10000, step:1000}
save_interval = 100 #@param {type:"slider", min:0, max:1000, step:100}
texture_resolution = 2048 #@param {type:"slider", min:0, max:4096, step:256}
batch_size = 12 #@param {type:"slider", min:0, max:20, step:4}
dmtet_grid = 64 #@param {type:"slider", min:0, max:128, step:4}
mesh_scale = 2.5 #@param {type:"slider", min:0, max:10, step:0.1}
camera_space_light = True #@param {type:"boolean"}

config = {
        "ref_mesh": ref_mesh,
        "random_textures": random_textures,
        "iter": iteration,
        "save_interval": save_interval,
        "texture_res": [texture_resolution, texture_resolution],
        "train_res": [train_resolution, train_resolution],
        "batch": batch_size,
        "learning_rate": [0.03, 0.01],
        "kd_min" : [0.03, 0.03, 0.03],
        "kd_max" : [0.8, 0.8, 0.8],
        "ks_min" : [0, 0.08, 0],
        "ks_max" : [0, 1.0, 1.0],
        "dmtet_grid" : dmtet_grid,
        "mesh_scale" : mesh_scale,
        "camera_space_light" : camera_space_light,
        "background" : "white",
        "display" : [{"bsdf":"kd"}, {"bsdf":"ks"}, {"bsdf" : "normal"}],
        "out_dir": f'{target_folder}_rescaled'
}

with open(config_path, 'w') as file:
    json.dump(config, file)

In [ ]:
# 学習スクリプトの実行
!python train.py --config $config_path

Config / Flags:
---------
config configs/Chair__rescaled.json
iter 3000
batch 12
spp 1
layers 1
train_res [512, 512]
display_res [512, 512]
texture_res [2048, 2048]
display_interval 0
save_interval 100
learning_rate [0.03, 0.01]
min_roughness 0.08
custom_mip False
random_textures True
background white
loss logl1
out_dir out/Chair__rescaled
ref_mesh data/nerd/Chair__rescaled
base_mesh None
validate True
mtl_override None
dmtet_grid 64
mesh_scale 2.5
env_scale 1.0
envmap None
display [{'bsdf': 'kd'}, {'bsdf': 'ks'}, {'bsdf': 'normal'}]
camera_space_light True
lock_light False
lock_pos False
sdf_regularizer 0.2
laplace relative
laplace_scale 10000.0
pre_load True
kd_min [0.03, 0.03, 0.03]
kd_max [0.8, 0.8, 0.8]
ks_min [0, 0.08, 0]
ks_max [0, 1.0, 1.0]
nrm_min [-1.0, -1.0, 0.0]
nrm_max [1.0, 1.0, 1.0]
cam_near_far [0.1, 1000.0]
learn_light True
local_rank 0
multi_gpu False
---------
DatasetLLFF: 21 images with shape [512, 512]
DatasetLLFF: auto-centering at [-1.139238   -0.06111449 18.4421

out下にフォルダが作成されるので、zip化してダウンロードしましょう。

In [ ]:
!zip -r out.zip out

Scanning files 
updating: out/ (stored 0%)
updating: out/nerd_custom/ (stored 0%)
updating: out/nerd_custom/img_dmtet_pass1_000000.png (deflated 1%)
updating: out/nerd_custom/img_dmtet_pass1_000001.png (deflated 2%)
updating: out/nerd_custom/img_dmtet_pass1_000002.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000003.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000004.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000005.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000006.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000007.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000008.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000009.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000010.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000011.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_000012.png (deflated 3%)
updating: out/nerd_custom/img_dmtet_pass1_00001